<a href="https://colab.research.google.com/github/martin-fabbri/advanced-react-components/blob/master/probabilistic-programming/variational_inference_pyro_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Variational Inference Pyro

In [0]:
!pip install -q pyro-ppl

In [0]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI
from pyro.infer import Trace_ELBO
from pyro.distributions import Beta
from pyro.distributions import Bernoulli

Symbol | Represents
:---:|---
$$\mathbf{z}$$	| The compressed code learned in the bottleneck layer
$$q_{\phi}(\mathbf{z}\vert\mathbf{x})$$	| Estimated posterior probability function
$$p_{\theta}(\mathbf{x}\vert\mathbf{z})$$ |	Likelihood of generating true data sample given the latent code

$$p_0(x, z) = p_o(x|z)p_0(z)$$

## Model

Learning a good model will be maximixing the log evidence:

$$\theta_\max=\underset{\theta}{argmax}\;{log\;p_0(x)}$$

where the log evidence $log\;p_{\theta}(x)$ is given by:

$$ log\;p_0(x) = log \int{dz\;p_{\theta}(x,z)} $$

Solving the integral over the latent random variable $\bf z$ is often intractable.  

In addition to finding $\theta_{\rm{max}}$, we would like to calculate the posterior over the latent variables $\bf z$:

$$ p_{\theta_{\rm{max}}}({\bf z} | {\bf x}) = \frac{p_{\theta_{\rm{max}}}({\bf x} , {\bf z})}{
\int \! d{\bf z}\; p_{\theta_{\rm{max}}}({\bf x} , {\bf z}) } $$

Note that the denominator of this expression is the (usually intractable) evidence. Variational inference offers a scheme for finding $\theta_{\rm{max}}$ and computing an approximation to the posterior $p_{\theta_{\rm{max}}}({\bf z} | {\bf x})$. Let's see how that works.

## Guide

The basic idea is that we introduce a parameterized distribution $q_{\phi}({\bf z})$, where $\phi$ are known as the variational parameters. This distribution is called the variational distribution in much of the literature, and in the context of Pyro it's called the **guide** (one syllable instead of nine!). The guide will serve as an approximation to the posterior.

> Note that **guide** does _not_ contain observed data, since the guide needs to be a properly normalized distribution.

FIX ME: Recall that when random variables are specified in Pyro with the primitive statement `pyro.sample()` the first argument denotes the name of the random variable. These names will be used to align the random variables in the model and guide. To be very explicit, if the model contains a random variable `z_1`

```python
def model():
    pyro.sample("z_1", ...)
```

then the guide needs to have a matching `sample` statement

```python
def guide():
    pyro.sample("z_1", ...)
```


## ELBO

Evidence lower bound (ELBO) is a function of both $\theta$ and $\phi$ defined as an expectation w.r.t. to samples from the **guide**:

$${\rm ELBO} \equiv \mathbb{E}_{q_{\phi}({\bf z})} \left [ 
\log p_{\theta}({\bf x}, {\bf z}) - \log q_{\phi}({\bf z})
\right]$$

By assumption we can compute the log probabilities inside the expectation. And since the guide is assumed to be a parametric distribution we can sample from, we can compute Monte Carlo estimates of this quantity. Crucially, the ELBO is a lower bound to the log evidence, i.e. for all choices of $\theta$ and $\phi$ we have that 

$$\log p_{\theta}({\bf x}) \ge {\rm ELBO} $$

So if we take (stochastic) gradient steps to maximize the ELBO, we will also be pushing the log evidence higher (in expectation). Furthermore, it can be shown that the gap between the ELBO and the log evidence is given by the KL divergence between the guide and the posterior:

$$ \log p_{\theta}({\bf x}) - {\rm ELBO} = 
\rm{KL}\!\left( q_{\phi}({\bf z}) \lVert p_{\theta}({\bf z} | {\bf x}) \right) $$

This KL divergence is a particular (non-negative) measure of 'closeness' between two distributions. So, for a fixed $\theta$, as we take steps in $\phi$ space that increase the ELBO, we decrease the KL divergence between the guide and the posterior, i.e. we move the guide towards the posterior. In the general case we take gradient steps in both $\theta$ and $\phi$ space simultaneously so that the guide and model play chase, with the guide tracking a moving posterior $\log p_{\theta}({\bf z} | {\bf x})$. Perhaps somewhat surprisingly, despite the moving target, this optimization problem can be solved (to a suitable level of approximation) for many different problems.

## Sample

You've been given a two-sided coin. You want to determine whether the coin is fair or not, i.e. whether it falls heads or tails with the same frequency. You have a prior belief about the likely fairness of the coin based on two observations:

- it's a standard quarter issued by the US Mint
- it's a bit banged up from years of use

So while you expect the coin to have been quite fair when it was first produced, you allow for its fairness to have since deviated from a perfect 1:1 ratio. So you wouldn't be surprised if it turned out that the coin preferred heads over tails at a ratio of 11:10. By contrast you would be very surprised if it turned out that the coin preferred heads over tails at a ratio of 5:1—it's not that banged up.

To turn this into a probabilistic model we encode heads and tails as 1s and 0s. We encode the fairness of the coin as a real number  f , where  f  satisfies  f∈[0.0,1.0]  and  f=0.50  corresponds to a perfectly fair coin. Our prior belief about  f  will be encoded by a beta distribution, specifically  Beta(10,10) , which is a symmetric probability distribution on the interval  [0.0,1.0]  that is peaked at  f=0.5 .

In [36]:
n_steps = 2000
pyro.enable_validation(True)

pyro.clear_param_store()

data = []
for _ in range(6):
  data.append(torch.tensor(1.0))

for _ in range(4):
  data.append(torch.tensor(0.0))

def model(data):
  alpha0 = torch.tensor(10.0)
  beta0 = torch.tensor(10.0)

  # sample f from the beta prior
  f = pyro.sample('latent_fairness', Beta(alpha0, beta0))
  for i in range(len(data)):
    pyro.sample(f'obs_{i}', Bernoulli(f), obs=data[i])

def guide(data):
  # register the two variational parameter with Pyro
  # - both parameters will have initial value 15.0
  # - because we invoke constraints.positive, the optimizer will
  #   take the gradients on the unconstrained parameters
  #   (which are related to the constrained parameters by a log)
  alpha_q = pyro.param('alpha_q', torch.tensor(15.0), 
                       constraint=constraints.positive)
  beta_q = pyro.param('beta_q', torch.tensor(15.0),
                      constraint=constraints.positive)
  pyro.sample('latent_fairness', Beta(alpha_q, beta_q))

adam_params = {'lr': 0.0005, 'betas': (0.90, 0.999)}
optimizer = Adam(adam_params)

svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

for step in range(n_steps):
  svi.step(data)
  if step % 100 == 0:
    print('.', end='')

# grag the learned variational parameters
alpha_q = pyro.param('alpha_q').item()
beta_q = pyro.param('beta_q').item()

# here we use some facts about the beta distribution
# compute the inferred mean of the coin's fairness
inferred_mean = alpha_q / (alpha_q + beta_q)

factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * math.sqrt(factor)

print(f'\nbased on the data and our prior belief, the fairness of the coin is {inferred_mean:.3f} +- {inferred_std:.3f}')


....................
based on the data and our prior belief, the fairness of the coin is 0.540 +- 0.090
